In [4]:
from matplotlib import pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import numpy as np
from time import time
import os

from utils.arquivo_utils import ArquivoUtils
from utils.enums.datasets_name_enum import DatasetName

# ==============================
# Função: aplicar Sobel numa imagem Torch
# ==============================
def aplicar_sobel(img_tensor: torch.Tensor) -> torch.Tensor:
    # Converter (C,H,W) -> (H,W,C)
    img = img_tensor.permute(1, 2, 0).numpy()
    img = (img * 255).astype(np.uint8)

    # Converter para escala de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Aplicar Sobel (X e Y)
    sobelX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=1)
    sobelY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=1)

    # Converter para uint8 e combinar
    sobelX = np.uint8(np.absolute(sobelX))
    sobelY = np.uint8(np.absolute(sobelY))
    sobel_combined = cv2.bitwise_or(sobelX, sobelY)

    # Converter para tensor normalizado [0,1]
    sobel_tensor = torch.from_numpy(sobel_combined).unsqueeze(0).float() / 255.0  # [1,H,W]
    return sobel_tensor


# ==============================
# Função: extrair e salvar features
# ==============================
def extrair_sobel_features(loader, nome_split="train"):
    feats, labels = [], []
    for i, (imgs, lbls) in enumerate(loader):
        # imgs: [B,C,H,W]
        sobel_batch = []
        for img in imgs:
            sobel_img = aplicar_sobel(img)
            sobel_batch.append(sobel_img)

        sobel_batch = torch.stack(sobel_batch)  # [B,1,H,W]
        B, C, H, W = sobel_batch.shape

        # Flatten (cada imagem vira um vetor 1D)
        sobel_flat = sobel_batch.view(B, -1).numpy()
        feats.append(sobel_flat)
        labels.append(lbls.numpy())

        print(f"\rProcessando {nome_split}: {i+1}/{len(loader)}", end="", flush=True)

    print()  # quebra linha
    X = np.concatenate(feats, axis=0)
    y = np.concatenate(labels, axis=0)
    print(f"{nome_split} -> {X.shape[0]} amostras, dimensão {X.shape[1]}")
    return X, y


# ==============================
# Main
# ==============================
if __name__ == "__main__":
    start = time()

    # CIFAR-10
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    trainset = torchvision.datasets.CIFAR10(root="./datasets", train=True, download=False, transform=transform)
    testset = torchvision.datasets.CIFAR10(root="./datasets", train=False, download=False, transform=transform)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=False)
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

    # Extrair Sobel features
    X_train, y_train = extrair_sobel_features(trainloader, "train")
    X_test, y_test = extrair_sobel_features(testloader, "test")

    ArquivoUtils.salvar_features_imagem(
			nome_tecnica_ext=f"sobel",
			nome_dataset=DatasetName.CIFAR10.value,
			dados_treino=X_train,
			classes_treino=y_train,
			dados_teste=X_test,
			classes_teste=y_test
		)

    end = time()
    print(f"\n✅ Concluído! Tempo total: {end - start:.2f}s")
    print("Arquivo salvo em: features/cifar10_sobel_features.npz")


Processando train: 391/391
train -> 50000 amostras, dimensão 1024
Processando test: 79/79
test -> 10000 amostras, dimensão 1024

✅ Concluído! Tempo total: 75.72s
Arquivo salvo em: features/cifar10_sobel_features.npz


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ==============================
# 1️⃣ Carregar uma imagem do CIFAR-10
# ==============================
transform = transforms.Compose([
    transforms.ToTensor()
])

# Carrega o dataset (não precisa baixar se já tiver)
dataset = torchvision.datasets.CIFAR10(root="./datasets", train=True, download=False, transform=transform)

# Escolher uma imagem de exemplo
img_tensor, label = dataset[8]
classes = dataset.classes
print(f"Classe: {classes[label]}")

# Converter para formato OpenCV (uint8, H×W×C)
img = img_tensor.permute(1, 2, 0).numpy()
img = (img * 255).astype(np.uint8)

# Converter para escala de cinza
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# ==============================
# 2️⃣ Aplicar Sobel com diferentes ksize
# ==============================
plt.figure(figsize=(12, 4))
plt.subplot(1, 4, 1)
plt.title("Original")
plt.imshow(img)
plt.axis('off')

for i, k in enumerate([1, 3, 5], start=2):
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=k)
    sobel = np.uint8(np.absolute(sobel))
    plt.subplot(1, 4, i)
    plt.title(f"Sobel ksize={k}")
    plt.imshow(sobel, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()
